In [197]:
using Revise
using PastaQ
using ITensors
using Random
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools
using LinearAlgebra
using JLD2
using Flux
using PyCall
using SymPy
using QOS

import mVQE
using mVQE: Circuits

using mVQE.Hamiltonians: hamiltonian_tfi, hamiltonian_ghz, hamiltonian_aklt_half
using mVQE.ITensorsExtension: projective_measurement
using mVQE: loss, optimize_and_evolve
using mVQE.Circuits: AbstractVariationalCircuit, VariationalCircuitRy, VariationalMeasurement, VariationalMeasurementMC, VariationalMeasurementMCFeedback
using mVQE.Misc: get_ancillas_indices, pprint
using mVQE.Optimizers: OptimizerWrapper
using mVQE.pyflexmps: pfs

In [2]:
singlet_gate(i1, i2)=[("X", i1), ("X", i2), ("H", i2), ("CX", (i2, i1))]


θ_1 = 2 * atan(-1 / sqrt(2))
θ_2 = 2 * atan(sqrt(2))

U_gate_L(c_in, i1, i2) =[("CX", (c_in, i1)), ("CRy", (i1, c_in), (θ = θ_1,)), ("X", i1),("CRy", (i1, i2), (θ = θ_2,)), ("X", i1),
                        ("CX", (c_in, i1)), ("CX", (i1, i2)), ("CX", (i2, i1))]

U_gate_R(c_in, i1, i2) =[("CX", (c_in, i1)), ("CRy", (i1, c_in), (θ = θ_1,)), ("X", i1),("CRy", (i1, i2), (θ = θ_2,)), ("X", i1),
                        ("CX", (c_in, i1)), ("CX", (i1, i2)), ("CX", (i2, i1)), ("SWAP", c_in, i1)]

U_gate_L_(i0, i1, i2) = U_gate_L(i2, i1, i0)
#U_gate_R(i3, i4, i5) = U_gate_R(i3, i4, i5)

U2_gate(i0, i1, i2, i3, i4, i5) = vcat(U_gate_L_(i0, i1, i2), U_gate_R(i3, i4, i5))
    
bell_gate(i0, i1) = [("CX", (i1, i0)), ("H", i1)]

bell_gate (generic function with 1 method)

In [3]:
Ux = zeros(2, 2, 2, 2)
Ux[1, 2, 2, 1] = 1
Ux[1, 1, 1, 1] = 1
Ux[2, 1, 1, 2] = 1
Ux[2, 2, 2, 2] = 1
Ux = reshape(Ux, 4, 4)
ITensors.op(::OpName"Ux", ::SiteType"Qubit") = Ux

Uy = zeros(2, 2, 2, 2)
Uy[1, 2, 2, 1] = -1 # |+><-|
Uy[1, 1, 1, 1] = 1 # |0><0|
Uy[2, 1, 1, 2] = -1 # |-><+|
Uy[2, 2, 2, 2] = 1
Uy = reshape(Uy, 4, 4)
ITensors.op(::OpName"Uy", ::SiteType"Qubit") = Uy

Uz = zeros(2, 2, 2, 2)
Uz[2, 1, 2, 1] = -1
Uz[1, 1, 1, 1] = 1
Uz[1, 2, 1, 2] = -1
Uz[2, 2, 2, 2] = 1
Uz = reshape(Uz, 4, 4)
ITensors.op(::OpName"Uz", ::SiteType"Qubit") = Uz

Ui = zeros(2, 2, 2, 2)
Ui[2, 1, 2, 1] = 1
Ui[1, 1, 1, 1] = 1
Ui[1, 2, 1, 2] = 1
Ui[2, 2, 2, 2] = 1
Ui = reshape(Ui, 4, 4)
ITensors.op(::OpName"Ui", ::SiteType"Qubit") = Ui

In [4]:
Uy*Uz

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0

In [5]:
Ux*Uz

4×4 Matrix{Float64}:
 1.0   0.0   0.0  0.0
 0.0   0.0  -1.0  0.0
 0.0  -1.0   0.0  0.0
 0.0   0.0   0.0  1.0

In [61]:
N_state = 8
state_indices, ancilla_indices, N = get_ancillas_indices(N_state, [false, true, true, true, true, false])
hilbert = qubits(N)

hilbert_state = hilbert[state_indices]
hilbert_ancilla = hilbert[ancilla_indices]

H = hamiltonian_ghz(state_indices, hilbert)
ψ = productstate(hilbert, fill(0, N))
state_indices, ancilla_indices, N

([2, 3, 4, 5, 8, 9, 10, 11], [1, 6, 7, 12], 12)

In [62]:
aklts = mVQE.StateFactory.AKLT_halfs(hilbert[state_indices], basis="girvin");
H_8, H_aklt_8, H_spin1_8 = hamiltonian_aklt_half(hilbert_state);
Htot_8, = hamiltonian_aklt_half(hilbert, sublattice=state_indices);

In [63]:
gates = vcat(singlet_gate(3, 4), singlet_gate(9, 10))
gates = vcat(gates, U2_gate(1, 2, 3, 4, 5, 6))
gates = vcat(gates, U2_gate(7, 8, 9, 10, 11, 12));
gates = vcat(gates, bell_gate(7, 6));

In [64]:
ψ2 = runcircuit(ψ, gates);

In [72]:
ψp, measurement = mVQE.ITensorsExtension.projective_measurement_sample(ψ2; indices=[1, 6, 7, 12], remove_measured=true)
#measurement = [2, 1, 2, 1]
#ψp = mVQE.ITensorsExtension.reduce_MPS(ψ2, [1, 6, 7, 12], measurement, norm=true)
println(inner(ψp, H_aklt_8, ψp))
println(inner(ψp, H_spin1_8, ψp))
measurement

-3.583441369784768e-16
-1.8676482051465933e-17


4-element Vector{Int64}:
 2
 2
 2
 1

In [73]:
correction_gates_ = Dict()

correction_gates_[[1, 1]] = "Uy"
correction_gates_[[1, 2]] = "Uz"
correction_gates_[[2, 1]] = "Ux"
correction_gates_[[2, 2]] = "Ui"

"Ui"

In [74]:
g = correction_gates_[measurement[2:3]]
gates = [(g, (1, 2)), (g, (3, 4))]
ψp_corr = runcircuit(ψp, gates);

g = correction_gates_[measurement[2:3]]
gates = [(g, (5, 6)), (g, (7, 8))]
ψp_corr2 = runcircuit(ψp, gates);

println(inner(ψp_corr, H_aklt_8, ψp_corr))
println(inner(ψp_corr, H_spin1_8, ψp_corr))

-1.520190956388908e-16
-1.7080666827577884e-17


In [75]:
[inner(ψp_corr, aklt)^2 for aklt in aklts]

4-element Vector{Float64}:
 1.0000000000000009
 5.722131837052231e-32
 4.937110032284334e-31
 0.0005948839976204658

In [76]:
[inner(ψp_corr2, aklt)^2 for aklt in aklts]

4-element Vector{Float64}:
 1.0000000000000009
 6.391228950878499e-32
 1.3375834148446364e-32
 0.0005948839976204646

## Larger AKLTs

In [77]:
N_state = 4 * 4
state_indices, ancilla_indices, N = get_ancillas_indices(N_state, [false, true, true, true, true, false])
hilbert = qubits(N)

hilbert_state = hilbert[state_indices]
hilbert_ancilla = hilbert[ancilla_indices]

ψ = productstate(hilbert, fill(0, N))
aklts = mVQE.StateFactory.AKLT_halfs(hilbert[state_indices], basis="girvin");
state_indices, ancilla_indices, N

([2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 23], [1, 6, 7, 12, 13, 18, 19, 24], 24)

In [78]:
aklts = mVQE.StateFactory.AKLT_halfs(hilbert[state_indices], basis="girvin")
H, = hamiltonian_aklt_half(hilbert_state)
Htot, Htot_aklt, Htot_spin1 = hamiltonian_aklt_half(hilbert, sublattice=state_indices);

In [79]:
function girvin_gates(N)
    gates = Vector()
    for site in 3:6:N
        gates = vcat(gates, singlet_gate(site, site + 1))
    end
    
    for site in 1:6:N
        gates = vcat(gates, U2_gate(site, site+1, site+2, site+3, site+4, site+5))
    end
    
    for site in 6:6:N-1
        gates = vcat(gates, bell_gate(site+1, site))
    end
    return gates
end
            
function correction_gates(M)
    M = M[2:end-1]
    @assert mod(length(M), 2) == 0
    M = reshape(M, (2, Int(length(M)/2)))
    gates = Vector()
    for i in 1:size(M, 2)
        Mi = M[:, i]
        g = correction_gates_[Mi]
        for sites in 1:4:4*i
            gates = vcat(gates, [(g, (sites, sites+1)), (g, (sites+2, sites+3))])
        end
    end
    return gates
end
            
function correction_gates2(M, sites)
    M = M[2:end-1]
    @assert mod(length(M), 2) == 0
    M = reshape(M, (2, Int(length(M)/2)))
    gates = Vector()
    for i in 1:size(M, 2)
        Mi = M[:, i]
        g = correction_gates_[Mi]
        for site in 1:4:4*i
            gates = vcat(gates, [(g, (sites[site], sites[site+1])),
                                 (g, (sites[site+2], sites[site+3]))])
        end
    end
    return gates
end

correction_gates2 (generic function with 1 method)

In [80]:
ψ2 = runcircuit(ψ, girvin_gates(N));

In [81]:
ψp, measurement = mVQE.ITensorsExtension.projective_measurement_sample(ψ2; indices=ancilla_indices, remove_measured=true);

ψp_corr = runcircuit(ψp, correction_gates(measurement));
inner(ψp_corr, H, ψp_corr)

-1.9963004004924863e-15

### Make the preparation a parametrized quantum circuit

In [82]:
singlet_gate_p(i1, i2, θ_1, θ_2) = [("Rx", i1, (θ=θ_1,)), ("Ry", i2, (θ=θ_2,)), ("CX", (i2, i1))]
singlet_gate_p_i(i1, i2) = singlet_gate_p(i1, i2, pi, -pi/2) 

θ_1 = 2 * atan(-1 / sqrt(2))
θ_2 = 2 * atan(sqrt(2))

U_gate_L_p(c_in, i1, i2, θ_1, θ_2, ϕ_1) =[("CX", (c_in, i1)), ("CRy", (i1, c_in), (θ = θ_1,)), ("Rx", i1, (θ=ϕ_1,)),("CRy", (i1, i2), (θ = θ_2,)), ("X", i1),
                        ("CX", (c_in, i1)), ("CX", (i1, i2)), ("CX", (i2, i1))]

U_gate_R_p(c_in, i1, i2, θ_1, θ_2, ϕ_1) =[("CX", (c_in, i1)), ("CRy", (i1, c_in), (θ = θ_1,)), ("Rx", i1, (θ=ϕ_1,)),("CRy", (i1, i2), (θ = θ_2,)), ("X", i1),
                        ("CX", (c_in, i1)), ("CX", (i1, i2)), ("CX", (i2, i1)), ("SWAP", c_in, i1)]

U2_gate_p(i0, i1, i2, i3, i4, i5, θ_1, θ_2, ϕ_1, θ_3, θ_4, ϕ_2) = vcat(U_gate_L_p(i2, i1, i0, θ_1, θ_2, ϕ_1), U_gate_R_p(i3, i4, i5, θ_3, θ_4, ϕ_2))
U2_gate_p_i(i0, i1, i2, i3, i4, i5) = U2_gate_p(i0, i1, i2, i3, i4, i5, θ_1, θ_2, pi, θ_1, θ_2, pi)
bell_gate(i0, i1) = [("CX", (i1, i0)), ("H", i1)]

bell_gate (generic function with 1 method)

In [83]:
op("H", hilbert[1]).tensor*op("X", hilbert[1]).tensor

Dim 1: (dim=2|id=610|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=610|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
  0.7071067811865475  0.7071067811865475
 -0.7071067811865475  0.7071067811865475

In [84]:
op("Ry", hilbert[1], θ=-pi/2).tensor

Dim 1: (dim=2|id=610|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=610|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
  0.7071067811865476  0.7071067811865475
 -0.7071067811865475  0.7071067811865476

In [85]:
struct GirvinCircuit2 <: AbstractVariationalCircuit
    params::Matrix{Float64}
    GirvinCircuit2(params::Matrix{Float64}) = new(params)
    GirvinCircuit2(N::Int) = new(2π .* rand(N, 8))
    GirvinCircuit2() = new(Matrix{Float64}(undef, 0, 0)) # Empty circuit to be used as a placeholder
end
function generate_circuit(model::GirvinCircuit2; params=nothing)
    if params === nothing
        params = model.params
    end
    
    gates = Vector()
    for (i, site) in enumerate(3:6:N)
        gates = vcat(gates, singlet_gate_p(site, site + 1, params[i, 1:2]...))
    end
    for (i, site) in enumerate(1:6:N)
            gates = vcat(gates, U2_gate_p(site, site+1, site+2, site+3, site+4, site+5, params[i, 3:end]...))
    end
    
    for site in 6:6:N-1
        gates = vcat(gates, bell_gate(site+1, site))
    end
    return gates
end

generate_circuit (generic function with 2 methods)

In [226]:
N

24

In [88]:
params = Array(hcat(fill([pi, -pi/2, θ_1, θ_2, pi, θ_1, θ_2, pi], Int(N_state/4))...)')
gcirc = GirvinCircuit2(params)
gates = generate_circuit(gcirc);

In [89]:
ψ2_2 = runcircuit(ψ, gates);

In [135]:
ψ2 = runcircuit(ψ, girvin_gates(N));

In [136]:
inner(ψ2_2, ψ2)

1.0000000000000002 - 2.302212852266881e-16im

### Make  the correction a parametrized quantum circuit

In [137]:
# Uz*Uz=I
# Ux*Ux=I
# Uy*Uy=I

# Ux*Uy=Uz
# Ux*Uz=Uy

# Uy*Uz=Ux

In [138]:
α, β, γ, δ1 = Sym("α, β, γ, δ")
U1 = [
exp(1im*(α-β-δ1)) * cos(γ) -exp(1im*(α-β+δ1)) * sin(γ)
exp(1im*(α+β-δ1)) * sin(γ)  exp(1im*(α+β+δ1)) * cos(γ)
]

α2, β2, γ2, δ2 = Sym("α_2, β_2, γ_2, δ_2")
U2 = [
exp(1im*(α2-β2-δ2)) * cos(γ2) -exp(1im*(α2-β2+δ2)) * sin(γ)
exp(1im*(α2+β2-δ2)) * sin(γ2)  exp(1im*(α2+β2+δ2)) * cos(γ)
]

2×2 Matrix{Sym}:
 exp(I*(α_2 - β_2 - δ_2))*cos(γ_2)  -exp(I*(α_2 - β_2 + δ_2))*sin(γ)
 exp(I*(α_2 + β_2 - δ_2))*sin(γ_2)   exp(I*(α_2 + β_2 + δ_2))*cos(γ)

In [139]:
U = U1 * U2

2×2 Matrix{Sym}:
 exp(I*(α - β - δ))*exp(I*(α_2 - β_2 - δ_2))*cos(γ)*cos(γ_2) - exp(I*(α - β + δ))*exp(I*(α_2 + β_2 - δ_2))*sin(γ)*sin(γ_2)  …  -exp(I*(α - β - δ))*exp(I*(α_2 - β_2 + δ_2))*sin(γ)*cos(γ) - exp(I*(α - β + δ))*exp(I*(α_2 + β_2 + δ_2))*sin(γ)*cos(γ)
 exp(I*(α + β - δ))*exp(I*(α_2 - β_2 - δ_2))*sin(γ)*cos(γ_2) + exp(I*(α + β + δ))*exp(I*(α_2 + β_2 - δ_2))*sin(γ_2)*cos(γ)               -exp(I*(α + β - δ))*exp(I*(α_2 - β_2 + δ_2))*sin(γ)^2 + exp(I*(α + β + δ))*exp(I*(α_2 + β_2 + δ_2))*cos(γ)^2

In [140]:
function ITensors.op(::OpName"U_rot", ::SiteType"Qubit"; α::Number, β::Number, γ::Number, δ::Number) 
    return [
    1 0 0 0
    0 exp(1im*(α-β-δ)) * cos(γ) -exp(1im*(α-β+δ)) * sin(γ)  0
    0 exp(1im*(α+β-δ)) * sin(γ)  exp(1im*(α+β+δ)) * cos(γ)  0
    0 0 0 1]
end

In [141]:
Ux

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0

In [142]:
correction_gates_params = Dict()
correction_gates_params[[1, 1]] = [pi/2, pi/2, pi/2, 0]
correction_gates_params[[1, 2]] = [0, 0, 0, pi]
correction_gates_params[[2, 1]] = [pi/2, pi/2, pi/2, pi]
correction_gates_params[[2, 2]] = [0, 0, 0, 0];

compare(X, Y) = sum(abs.(X - Y)) < 1e-5
compare(X, Y, tX, tY) = (compare(X, tX) && compare(Y, tY)) || (compare(X, tY) && compare(Y, tX))

function add(a, b)
    if compare(a, b)
        out = zeros(4)

    elseif compare(a, [0,0,0,0]) || compare(b, [0,0,0,0])
        out = a .+ b

    elseif compare(a, b, [pi/2,pi/2,pi/2,0], [pi/2,pi/2,pi/2,pi])
        out = [0,0,0,pi]

    elseif compare(a, b, [pi/2,pi/2,pi/2,0], [0,0,0,pi])
        out = [pi/2,pi/2,pi/2,pi]

    elseif compare(a, b, [0,0,0,pi], [pi/2,pi/2,pi/2,pi])
        out = [pi/2,pi/2,pi/2,0]
    else
        println(compare(a, b, [pi/2,pi/2,pi/2,0], [0,0,0,pi]))
        @assert false "$a $b"
    end
    return out
end

function param_correction_gates(M)
    Ns_spin1 = length(M)
    M = M[2:end-1]
    
    @assert mod(length(M), 2) == 0
    
    M = reshape(M, (2, Int(length(M)/2)))
    angles = zeros((4, Ns_spin1))
    for i in 1:size(M, 2)
        Mi = M[:, i]
        g = correction_gates_params[Mi]
        for j in 1:2i
            angles[:, j] = add(angles[:, j], g)
        end
    end
    return angles
end
params = param_correction_gates(measurement)

4×8 Matrix{Float64}:
 0.0      0.0      0.0  0.0  1.5708   1.5708   0.0  0.0
 0.0      0.0      0.0  0.0  1.5708   1.5708   0.0  0.0
 0.0      0.0      0.0  0.0  1.5708   1.5708   0.0  0.0
 3.14159  3.14159  0.0  0.0  3.14159  3.14159  0.0  0.0

In [143]:
struct GirvinCorrCircuit2 <: AbstractVariationalCircuit end
function generate_circuit(model::GirvinCorrCircuit2; params=nothing)
    @assert size(params, 1) == 4
    N = size(params, 2)
    N_state = N * 2
    
    state_indices, = get_ancillas_indices(N_state, [false, true, true, true, true, false])
    println(state_indices)
    gates = Vector()
    
    for site in 1:N
        p = (α=params[1, site], β=params[2, site], γ=params[3, site], δ=params[4, site])
        gates = vcat(gates, [("U_rot", (state_indices[2site-1], state_indices[2site]), p)])
    end
    return gates
end
circ = GirvinCorrCircuit2()


GirvinCorrCircuit2()

In [144]:
# Fix the gates

In [189]:
ψp, measurement = mVQE.ITensorsExtension.projective_measurement_sample(ψ2; indices=ancilla_indices);

params = param_correction_gates(measurement)
gates = generate_circuit(circ; params)
gates2 = correction_gates2(measurement, state_indices)
ψp_corr = runcircuit(ψp, gates);
inner(ψp_corr, Htot, ψp_corr)

[2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 23]


-2.3178148509082522e-15 - 3.447678710060339e-33im

In [146]:
gates = generate_circuit(circ; params);

[2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 23]


In [147]:
ψp_corr = runcircuit(ψp, gates2);
inner(ψp_corr, Htot, ψp_corr)

-2.2322903936280042e-15

In [148]:
ψp_corr = runcircuit(ψp, gates);
inner(ψp_corr, Htot, ψp_corr)

-2.5967250751520253e-15 - 9.180444230631619e-33im

In [149]:
t = op("U_rot", hilbert[1], hilbert[2] ; α = pi, β = pi, γ = pi, δ = pi)
real(reshape(t.tensor, 4, 4))

Dim 1: 4
Dim 2: 4
NDTensors.Dense{Float64, Vector{Float64}}
 4×4
 1.0   0.0  0.0  0.0
 0.0   1.0  0.0  0.0
 0.0  -0.0  1.0  0.0
 0.0   0.0  0.0  1.0

## Everithing combined

In [162]:
gcirc = mVQE.GirvinProtocol.GirvinCircuitIdeal(N_state);
corrcirc = mVQE.GirvinProtocol.GirvinCorrCircuit();

In [264]:
ψ2_ = gcirc(ψ);
ψp_, measurement = mVQE.ITensorsExtension.projective_measurement_sample(ψ2_; indices=ancilla_indices)
params = mVQE.GirvinProtocol.param_correction_gates(measurement .- 1);
ψp_corr_ = corrcirc(ψp_; params)
inner(ψp_corr_, Htot, ψp_corr_)

LoadError: MethodError: no method matching param_correction_gates(::Vector{Int64})